In [2]:
import os
from pyspark.sql.session import SparkSession
from pathlib import Path

In [5]:
absolatepath = Path().absolute()
print(absolatepath)

/home/glue_user/workspace/jupyter_workspace/external_pra/eg-0001/program 0001


In [6]:
inputcsv = os.path.join(absolatepath,'input.csv')

In [10]:
spark = SparkSession.builder.master("local[2]").appName("pr0001").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 05:04:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/01 05:04:13 WARN 

In [12]:
sparkdf = spark.read.csv(inputcsv,header=True,inferSchema=True)
sparkdf.show()

+-----+---+------+
| name|age|gender|
+-----+---+------+
| Paul| 40|  male|
| Paul| 38|  male|
|David| 15|  male|
|Susan| 40|female|
|Susan| 34|female|
+-----+---+------+



In [13]:
sparkdf.createOrReplaceTempView("sqlscript")

In [14]:
filterData = spark.sql("select name,avg(age) from sqlscript group by name having count(name) > 1")
filterData.show()

+-----+--------+
| name|avg(age)|
+-----+--------+
|Susan|    37.0|
| Paul|    39.0|
+-----+--------+



In [16]:
outputpath = os.path.join(absolatepath,"output")

In [18]:
filterData.repartition(1).write.mode("overwrite").json(outputpath)